# Классификация комментариев с BERT и TF-IDF

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Цели:
* Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
* Постройте модель со значением метрики качества *F1* не меньше 0.75.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import re
import pickle
from tqdm import tqdm, notebook
tqdm.pandas(desc="progress bar!")

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

import torch
import transformers
from transformers import AutoModel, AutoTokenizer 

from catboost import CatBoostClassifier, Pool, cv

from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
nltk.download('popular', quiet=True);

In [3]:
pd.options.display.float_format = '{:0.3f}'.format
pd.options.mode.chained_assignment = None

%config InlineBackend.figure_format = 'retina'

Загрузка комментариев.

In [4]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')
df.shape

(159571, 2)

Оценим сбалансированность выборки.

* Токсичных комментариев намного меньше.
* Присутствует дисбаланс классов.

In [5]:
df.toxic.value_counts() / df.toxic.size

0   0.898
1   0.102
Name: toxic, dtype: float64

Очистим текст при помощи регулярных выражений. Это впоследствии пригодится для tf-idf с CatBoost и поможет провести проверку на дубликаты. 

In [6]:
# Ограничим число символов - 5000. Этого достаточно для задачи и поможет увеличить скорость расчетов.
def clear_text(text, char_limit=5000):
    # Приведем текст к нижнему регистру очистив его от лишних символов и цифр.
    text = re.sub(r"[^a-z']", ' ', text.lower())
    
    # Обрежим сообщение до ближайшего целого слова.
    try:
        r = re.match(r'(?P<truncated> .+) \s', text[:char_limit], re.X)
        text = r['truncated'].split()
        return ' '.join(text)
    except:
        return 'empty line'

In [7]:
# Датафрейм для tf-idf с CatBoost.
df2vec = df.copy()

In [8]:
df2vec['text'] = df2vec['text'].map(clear_text).copy()
df2vec.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d'aww he matches this background colour i'm se...,0
2,hey man i'm really not trying to edit war it's...,0
3,more i can't make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0


Теперь проверим дубликаты и удалим их.

In [9]:
df2vec['text'].duplicated().sum()

1356

In [10]:
df = df[~df2vec['text'].duplicated()]
df2vec = df2vec[~df2vec['text'].duplicated()]

print(df.shape, df2vec.shape)

(158215, 2) (158215, 2)


Обработаем текст при помощи лемматизатора Wordnet из NLTK.

In [11]:
# Инициализируем WordNetLemmatizer.
lemmatizer = WordNetLemmatizer()

# Функция для определения формы речи перед лемматизацией.
def get_wordnet_pos(word):
    # nltk.pos_tag() возвращает кортеж с тегом POS.
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# Функция лемматизации.
def lemmatize(text):
    return ' '.join(
        [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    )

Лемматизация процесс долгий. Воспользуемся pickle для сохранения и загрузки её результатов.

In [12]:
try:
    with open('lemmatized_test.pickle', 'rb') as f:
        df2vec['text'] = pickle.load(f)
except:
    df2vec['text'] = df2vec['text'].progress_apply(lemmatize)
    with open('lemmatized_test.pickle', 'wb') as f:
        pickle.dump(df2vec['text'], f)

Добавим два новых признака: число уникальных слов и количество символов в тексте.

In [13]:
df2vec['comment_rich'] = df2vec['text'].str.split().map(lambda x: len(set(x)))
df2vec['comment_len'] = df2vec['text'].map(len)

# Обучение

Проведем последовательное обучением различных моделей для получения более подробного отчета.

## CatBoostClassifier

Разделим выборку и сформируем Pool'ы для CatBoost.

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    df2vec.drop('toxic', axis=1),
    df2vec['toxic'],
    test_size=0.2,
    stratify=df2vec['toxic'],
    random_state=38)

train_set = Pool(X_train, y_train, text_features=['text'])

Проведем кросс-валидацию.

In [15]:
# Гиперпараметры.
params = {
    'objective': 'Logloss',
    'loss_function': 'Logloss',
    'task_type': 'GPU',
    'devices': '0:1',
    'depth': 6,
    'scale_pos_weight': 1.5,
    'random_strength': 0.2,
    'l2_leaf_reg': 4,
}

In [16]:
cv_data = cv(
    pool=train_set,
    params=params,
    early_stopping_rounds=100,
    logging_level='Silent',
    fold_count=3,
    stratified=True,
    partition_random_seed=38
)

# Результаты кросс-валидации.
cv_data.iloc[-1, :]

iterations           999.000
test-Logloss-mean      0.142
test-Logloss-std       0.004
train-Logloss-mean     0.130
train-Logloss-std      0.003
Name: 999, dtype: float64

Проверим метрики на тестовой выборке.

In [17]:
cb = CatBoostClassifier(random_state=38, **params)
cb.fit(train_set, verbose=250)

print()
print(classification_report(y_test, cb.predict(X_test)))

0:	learn: 0.6515623	total: 58.9ms	remaining: 58.9s
250:	learn: 0.1569367	total: 2.51s	remaining: 7.49s
500:	learn: 0.1459845	total: 4.82s	remaining: 4.8s
750:	learn: 0.1398743	total: 7.13s	remaining: 2.36s
999:	learn: 0.1353803	total: 9.43s	remaining: 0us

              precision    recall  f1-score   support

           0       0.97      0.98      0.98     28424
           1       0.85      0.76      0.80      3219

    accuracy                           0.96     31643
   macro avg       0.91      0.87      0.89     31643
weighted avg       0.96      0.96      0.96     31643



## TF-IDF

In [18]:
# Подгрузим стоп слова.
nltk_stopwords = stopwords.words('english')

ct = ColumnTransformer(
    # Векторизация TF-IDF текстовых данных.
    [('text', TfidfVectorizer(max_df=0.85,
                              # stop_words = nltk_stopwords,
                              max_features=6000), 'text')],
    # Остальные признаки оставляем как есть.
    remainder='passthrough',
    verbose_feature_names_out=False)

In [19]:
X_train_vec = ct.fit_transform(X_train)
X_test_vec = ct.transform(X_test)

print(X_train_vec.shape, X_test_vec.shape)

(126572, 6002) (31643, 6002)


Посчитаем основные метрики полученные у LogisticRegression.

In [20]:
lr = LogisticRegression(max_iter=1000,
                        class_weight={0: 1, 1: 1.5},
                        n_jobs=-1,
                        C=6,
                        random_state=38)
lr.fit(X_train_vec, y_train)
print(classification_report(y_test, lr.predict(X_test_vec)))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98     28424
           1       0.84      0.76      0.80      3219

    accuracy                           0.96     31643
   macro avg       0.91      0.87      0.89     31643
weighted avg       0.96      0.96      0.96     31643



## BERT - unitary/toxic-bert model

In [21]:
torch.cuda.is_available()

True

Будем использовать AutoModel и AutoTokenizer из библиотеки transformers.

In [22]:
# Зададим модель, она как раз заточена под toxic comments.
model_name = 'unitary/toxic-bert'

# Загрузим предобученную PyTorch модель.
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Модель будем просчитвать используюя GPU.
torch.device('cuda')
model.to('cuda:0');

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Отредактируем настройки, в частности поменяем problem_type с мультиклассовой классификации.

In [23]:
model.config = model.config.from_dict(
    {
        'id2label': {'0': 'not_toxic', '1': 'toxic'},
        'label2id': {'not_toxic': '0', 'toxic': '1'},
        'problem_type': 'single_label_classification',
    }
)

Преобразуем данные. На выходе получим input tokens и маску значимости.

In [24]:
input_ids, _, attention_mask = tokenizer(df['text'].values.tolist(),
                                         return_tensors='pt',
                                         max_length=384, # ограничим длину, иначе всё очень долго считается.
                                         padding=True, 
                                         truncation=True).values()

Сформируем окончательные признаки.

In [25]:
try:
    with open('features_full.pickle', 'rb') as f:
        features = pickle.load(f)    
except:
    # Количесто батчей чтобы не забить всю память расчетами.
    batch_size = 100
    size = df.shape[0]
    embeddings = []

    for i in notebook.tqdm(range(size // batch_size + (size % batch_size != 0))):
        with torch.no_grad():
            outputs = model(input_ids[batch_size*i:batch_size*(i+1)].cuda(),
                            attention_mask=attention_mask[batch_size*i:batch_size*(i+1)].cuda())
            # Найдем среднее в выходном тензоре.
            outputs_mean = torch.mean(outputs[0], dim=1)
            embeddings.append(outputs_mean)
            
    # Выгружаем наши данные обратно на CPU.
    features = np.concatenate([x.cpu() for x in embeddings])
    
    # Сохраняем их.
    with open('features_full.pickle', 'wb') as f:
        pickle.dump(features, f)

In [26]:
features.shape

(158215, 768)

Разбивка навыборки.

In [27]:
X_train, X_test, y_train, y_test = \
    train_test_split(features, df['toxic'], test_size=0.2, random_state=38, stratify=df['toxic'])

Предсказание модели.

In [28]:
lr = LogisticRegression(n_jobs=-1)
lr.fit(X_train, y_train)
print(classification_report(y_test, lr.predict(X_test)))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     28424
           1       0.95      0.95      0.95      3219

    accuracy                           0.99     31643
   macro avg       0.97      0.97      0.97     31643
weighted avg       0.99      0.99      0.99     31643



In [29]:
cb.fit(X_train, y_train, verbose=250)

print()
print(classification_report(y_test, cb.predict(X_test)))

0:	learn: 0.6129486	total: 11.6ms	remaining: 11.5s
250:	learn: 0.0324055	total: 2.94s	remaining: 8.79s
500:	learn: 0.0274462	total: 5.7s	remaining: 5.68s
750:	learn: 0.0239816	total: 8.43s	remaining: 2.79s
999:	learn: 0.0212323	total: 11.2s	remaining: 0us

              precision    recall  f1-score   support

           0       1.00      0.99      0.99     28424
           1       0.94      0.96      0.95      3219

    accuracy                           0.99     31643
   macro avg       0.97      0.97      0.97     31643
weighted avg       0.99      0.99      0.99     31643



# Выводы

* Получена модель, классифицирующая комментарии на позитивные и негативные.
* Достигнута f1 ~ 0.95 на тестовой выборке благодаря правильно выбранной модели unitary/toxic-bert.
* Проведено сравнение BERT, CatBoostClassifier и TF-IDF based.
    * Если доступно много ресурсов и нужна повышенная точность, то лучше всего использовать трасформеры.
    * Если требуется быстрый результат из коробки, то побеждает CatBoostClassifier.
* Векторизация и последующая классификация TF-IDF показала себя неплохо, но возможно следует выбрать другой классификатор для достижения более высоких результатов.